In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import yaml
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix, vstack
from scipy.sparse.linalg import svds
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MinMaxScaler, normalize

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ndcg_score, average_precision_score, precision_score, recall_score

import spotipy 
from spotipy.oauth2 import SpotifyOAuth
from surprise import NMF, SVD, SVDpp, KNNBasic, KNNWithMeans, KNNWithZScore, CoClustering
from surprise.model_selection import cross_validate
from surprise import Reader, Dataset

import scipy.sparse as sp


from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from mlxtend.evaluate import bias_variance_decomp

from tqdm import tqdm

import os
import sys
import surprise
import scrapbook as sb
import pandas as pd

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions

from collections import defaultdict

from surprise import Dataset, SVD
from surprise.model_selection import KFold

from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

from typing import List
from math import sqrt
import itertools


In [2]:
#POTREBNO IZMJENIT PUTANJU!!!!
#PREUZIMANJE PODATAKA GENERIRANIH IZ SPOTIFY

playlist_tracks_df_1 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars4\sars4\spotify\playlist_tracks.pkl")
playlist_tracks_df_2 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars8\spotify\playlist_tracks_2.pkl")
playlist_tracks_df_3 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars10\spotify\playlist_tracks_3.pkl")
playlist_tracks_df = pd.concat([playlist_tracks_df_1,playlist_tracks_df_2, playlist_tracks_df_3], ignore_index=True)

In [3]:
playlist_tracks_df['event_strength'] = 1  

In [6]:
#DEFINIRANJE DATAFRAMEA GDJE JE DEFINIRAN SVAKI PAR (KORISNIK-STVAR)->OCJENA

playlist_tracks_df['event_strength'] = 0  
user_set = playlist_tracks_df['playlist_id'].unique()
item_set=playlist_tracks_df['id'].unique()
ptm = pd.DataFrame(columns=["userID", "itemID", "rating"])

In [7]:
#KOMBINIRAJ PAROVE KORISNIK-STVAR U DATAFRAME OBLIKA-(KORISNIK-STVAR->AKO JE KORISNIK IMAO INTERAKCIJU SA STVARI - 1, ELSE - 0)

ptm_dict={}
for user in tqdm(user_set):
    user_item_proto=playlist_tracks_df[playlist_tracks_df['playlist_id']==user]['id']
    ptm_dict[user]=pd.DataFrame(pd.Series({"userID": user, "itemID": item, "rating": 1}) 
                    if(user_item_proto.str.contains(item, case=False).any())
                    else (pd.Series({"userID": user, "itemID": item, "rating": 0}))
                    for item in item_set)

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [30:41<00:00, 12.36s/it]


In [8]:
#SPREMI GENERIRANI DICTIONARY U ZAJEDNICKI DATAFRAME

ptm=pd.DataFrame(columns=["userID", "itemID", "rating"])
for elem in tqdm(ptm_dict):
    if len(ptm)==0:
        ptm=ptm_dict[elem]
    else:
        ptm = pd.concat([ptm, ptm_dict[elem]], ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:03<00:00, 41.47it/s]


In [9]:
#DEFINIRAJ READER OD 0-1 S KOJIM CE SE PTM PRENIJET NA DATA
#DATA SE UČI NA ALGORITMU KNNBasic()

reader = Reader(rating_scale=(0,1))
data = Dataset.load_from_df(ptm, reader)
algo = KNNBasic()
algo.fit(data.build_full_trainset())
my_recs = []

Computing the msd similarity matrix...
Done computing similarity matrix.


In [10]:
#RADI PREDVIĐANJA

results_KNN_dict={}
for user in tqdm(user_set):
    results_KNN_dict[user]=pd.DataFrame(pd.Series({"userID": user, "itemID": item, "pred": algo.predict(uid=user,iid=item).est})
                                                for item in item_set)

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [18:21<00:00,  7.39s/it]


In [11]:
#PRENESI PREDVIĐANJA IZ DICTIONARYJA U ZAJEDNIČKI DATAFRAME

results_KNN=pd.DataFrame(columns=["userID", "itemID", "rating"])
for elem in tqdm(results_KNN_dict):
    if len(results_KNN)==0:
        results_KNN=results_KNN_dict[elem]
    else:
        results_KNN = pd.concat([results_KNN, results_KNN_dict[elem]], ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:02<00:00, 66.99it/s]


In [14]:
#IZRACUNAJ PROSJECNU VRIJEDNOST STOTOG ELEMENTA

average_100th_list=[]
for user in tqdm(results_KNN_dict):
    average_100th_list.append(results_KNN_dict[user]['pred'].sort_values(ascending=False).iloc[99])
average_100th=sum(average_100th_list)/len(average_100th_list)
print("AVERAGE 100TH:  ",average_100th)

100%|██████████████████████████████████████████████████████████████████████████████| 149/149 [00:00<00:00, 1733.11it/s]

AVERAGE 100TH:   0.04102484896153723


In [15]:
#Izračunaj koliko se koja pjesma ponavlja u preporukama za sve korisnike

preds_dict_rec_100={}
items_count_dict={}
for item in playlist_tracks_df.index:
    items_count_dict[item]=0

for user in tqdm(results_KNN_dict):
    preds_dict_rec_100[user]=results_KNN_dict[user]['pred'].sort_values(ascending=False).head(99)
    for item in preds_dict_rec_100[user].index:
        items_count_dict[item]+=1

100%|██████████████████████████████████████████████████████████████████████████████| 149/149 [00:00<00:00, 1263.13it/s]


In [16]:
#SPAJANJE MATRICA USER-ITEM-RATING-PREDICTION

ui_rating_prediction_df = ptm.merge(results_KNN, on=['userID', 'itemID'], how='inner')

In [17]:
print("CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-")

CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-


In [26]:
average_rmse=cross_validate(algo, data, measures=['RMSE'], cv=10, verbose=True)
print('CV CROSS_VALIDATE RMSE: ' + str(sum(average_rmse['test_rmse']) / len(average_rmse['test_rmse'])))

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.0983  0.0998  0.1006  0.0991  0.0984  0.0990  0.0961  0.0990  0.0995  0.0989  0.

In [27]:
average_mae=cross_validate(algo, data, measures=['MAE'], cv=10, verbose=True)
print('CV CROSS_VALIDATE MAE: ' + str(sum(average_mae['test_mae']) / len(average_mae['test_mae'])))

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
MAE (testset)     0.0130  0.0128  0.0129  0.0128  0.0126  0.0130  0.0135  0.0131  0.0135  0.0129  0.0

In [17]:
kf = KFold(n_splits=10)
algo_test = KNNBasic()
map_results_cv = []
ndcg_results_cv = []
rmse_list_cv2=[]
mae_list_cv2=[]

precision_results_cv_micro = []
precision_results_cv_macro = []
precision_results_cv_weighted = []
recall_results_cv_micro = []
recall_results_cv_macro = []
recall_results_cv_weighted = []

for trainset, testset in tqdm(kf.split(data)):
    
    #SAMO ZA JEDNOG ILI ZA SVE KORISNIKE
    
    algo_test.fit(trainset)                   #UCENJE
    predictions = algo_test.test(testset)     #PREDVIDANJE
    dframe = pd.DataFrame(predictions, columns=["uid", "iid", "true_r", "est", "details"])
    
    #EVALUACIJA
    
    rmse_list_cv2.append(sqrt(mean_squared_error(dframe['est'], dframe['true_r'])))
    mae_list_cv2.append((mean_absolute_error(dframe['est'], dframe['true_r'])))
    
    average_map = average_precision_score(dframe['true_r'].values, dframe['est'].values)
    map_results_cv.append(average_map)
        
    average_ndcg=ndcg_score(dframe['true_r'].values.reshape(1, -1), dframe['est'].values.reshape(1, -1))
    ndcg_results_cv.append(average_ndcg)
    
    df=dframe.copy()
    df.loc[df['est']>=average_100th, 'est']=1
    df.loc[df['est']<average_100th, 'est']=0
    
    precision_results_cv_micro.append(precision_score(df['true_r'], df['est'], average='micro'))
    recall_results_cv_micro.append(recall_score(df['true_r'], df['est'], average='micro'))
    
    precision_results_cv_macro.append(precision_score(df['true_r'], df['est'], average='macro'))
    recall_results_cv_macro.append(recall_score(df['true_r'], df['est'], average='macro'))
    
    precision_results_cv_weighted.append(precision_score(df['true_r'], df['est'], average='weighted'))
    recall_results_cv_weighted.append(recall_score(df['true_r'], df['est'], average='weighted'))

mean_rmse_score_cv=np.mean(rmse_list_cv2)
mean_mae_score_cv=np.mean(mae_list_cv2)    
mean_map_score_cv = np.mean(map_results_cv)
mean_ndcg_score_cv = np.mean(ndcg_results_cv)

mean_precision_score_cv_micro=np.mean(precision_results_cv_micro)
mean_recall_score_cv_micro=np.mean(recall_results_cv_micro)

mean_precision_score_cv_macro=np.mean(precision_results_cv_macro)
mean_recall_score_cv_macro=np.mean(recall_results_cv_macro)

mean_precision_score_cv_weighted=np.mean(precision_results_cv_weighted)
mean_recall_score_cv_weighted=np.mean(recall_results_cv_weighted)

f1_score_cv_micro=(2*mean_precision_score_cv_micro*mean_recall_score_cv_micro)/(mean_precision_score_cv_micro+mean_recall_score_cv_micro)
f1_score_cv_macro=(2*mean_precision_score_cv_macro*mean_recall_score_cv_macro)/(mean_precision_score_cv_macro+mean_recall_score_cv_macro)
f1_score_cv_weighted=(2*mean_precision_score_cv_weighted*mean_recall_score_cv_weighted)/(mean_precision_score_cv_weighted+mean_recall_score_cv_weighted)

print("CV SKLEARN.METRICS RMSE:    ",mean_rmse_score_cv)
print("CV SKLEARN.METRICS MAE:    ",mean_mae_score_cv)
print("CV SKLEARN.METRICS MAP:    ", mean_map_score_cv)
print("CV SKLEARN.METRICS NDCG:    ", mean_ndcg_score_cv)
print("CV SKLEARN.METRICS PRECISION:    ",mean_precision_score_cv_micro,"|",
                      mean_precision_score_cv_macro, "|", mean_precision_score_cv_weighted)
print("CV SKLEARN.METRICS RECALL:    ",mean_recall_score_cv_micro,"|",
                      mean_recall_score_cv_macro, "|", mean_recall_score_cv_weighted)
print("CV SKLEARN.METRICS F1:   ", f1_score_cv_micro,"|",
                      f1_score_cv_macro, "|", f1_score_cv_weighted)

0it [00:00, ?it/s]

Computing the msd similarity matrix...
Done computing similarity matrix.


1it [00:30, 30.31s/it]

Computing the msd similarity matrix...
Done computing similarity matrix.


2it [01:00, 30.26s/it]

...

Computing the msd similarity matrix...
Done computing similarity matrix.


9it [08:53, 74.34s/it]

Computing the msd similarity matrix...
Done computing similarity matrix.


10it [10:08, 60.81s/it]

CV SKLEARN.METRICS RMSE:     0.09887553367023426
CV SKLEARN.METRICS MAE:     0.012978723476730597
CV SKLEARN.METRICS MAP:     0.037102587416326355
CV SKLEARN.METRICS NDCG:     0.6181540194540517
CV SKLEARN.METRICS PRECISION:     0.9744437223556288 | 0.5331369800598487 | 0.9821965098925025
CV SKLEARN.METRICS RECALL:     0.9744437223556288 | 0.5609414981959416 | 0.9744437223556288
CV SKLEARN.METRICS F1:    0.9744437223556288 | 0.5466859321009565 | 0.9783047567049958


In [18]:
print("TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-")

TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-


In [19]:
trainset_tt, testset_tt = train_test_split(data, test_size=0.2, random_state=42)

algo_tt = KNNBasic()
algo_tt.fit(trainset_tt)                     #UCENJE
predictions_tt = algo_tt.test(testset_tt)    #PREDVIĐANJE NA TEST DATA

dframe_tt = pd.DataFrame(predictions_tt, columns=["uid", "iid", "true_r", "est", "details"])

#EVALUACIJA

average_rmse_TT=sqrt(mean_squared_error(dframe_tt['est'], dframe_tt['true_r']))
average_mae_TT=(mean_absolute_error(dframe_tt['est'], dframe_tt['true_r']))
average_ndcg_tt=ndcg_score(dframe_tt['true_r'].values.reshape(1, -1), dframe_tt['est'].values.reshape(1, -1))
average_map_tt=average_precision_score(dframe_tt['true_r'].values, dframe_tt['est'].values)

print('TRAIN-TEST SKLEARN.METRICS RMSE: ' + str(average_rmse_TT))
print('TRAIN-TEST SKLEARN.METRICS MAE: ' + str(average_mae_TT))
print('TRAIN-TEST SKLEARN.METRICS NDCG: ' + str(average_ndcg_tt))
print('TRAIN-TEST SKLEARN.METRICS MAP: ' + str(average_map_tt))

df_tt=dframe_tt.copy()
df_tt.loc[df_tt['est']>=average_100th, 'est']=1
df_tt.loc[df_tt['est']<average_100th, 'est']=0

average_precision_tt_micro=(precision_score(df_tt['true_r'], df_tt['est'], average='micro'))
average_precision_tt_macro=(precision_score(df_tt['true_r'], df_tt['est'], average='macro'))
average_precision_tt_weighted=(precision_score(df_tt['true_r'], df_tt['est'], average='weighted'))

average_recall_tt_micro=(recall_score(df_tt['true_r'], df_tt['est'], average='micro'))
average_recall_tt_macro=(recall_score(df_tt['true_r'], df_tt['est'], average='macro'))
average_recall_tt_weighted=(recall_score(df_tt['true_r'], df_tt['est'], average='weighted'))

average_f1_tt_micro=(2*average_precision_tt_micro*average_recall_tt_micro)/(average_precision_tt_micro+average_recall_tt_micro)
average_f1_tt_macro=(2*average_precision_tt_macro*average_recall_tt_macro)/(average_precision_tt_macro+average_recall_tt_macro)
average_f1_tt_weighted=(2*average_precision_tt_weighted*average_recall_tt_weighted)/(average_precision_tt_weighted+average_recall_tt_weighted)


print("TRAIN-TEST SKLEARN.METRICS PRECISION:    ", average_precision_tt_micro, "|",average_precision_tt_macro, "|",average_precision_tt_weighted)
print("TRAIN-TEST SKLEARN.METRICS RECALL:    ", average_recall_tt_micro,"|",average_recall_tt_macro,"|",average_recall_tt_weighted)   
print("TRAIN-TEST SKLEARN.METRICS F1:   ",average_f1_tt_micro ,"|", average_f1_tt_macro,"|", average_f1_tt_weighted)

Computing the msd similarity matrix...
Done computing similarity matrix.
TRAIN-TEST SKLEARN.METRICS RMSE: 0.09941102803969593
TRAIN-TEST SKLEARN.METRICS MAE: 0.013430588558381062
TRAIN-TEST SKLEARN.METRICS NDCG: 0.6350113852030178
TRAIN-TEST SKLEARN.METRICS MAP: 0.03266813056636299
TRAIN-TEST SKLEARN.METRICS PRECISION:     0.9725825793088412 | 0.5307288463932868 | 0.9820160579274213
TRAIN-TEST SKLEARN.METRICS RECALL:     0.9725825793088412 | 0.5615294280721956 | 0.9725825793088412
TRAIN-TEST SKLEARN.METRICS F1:    0.9725825793088412 | 0.5456948645639333 | 0.9772765542210955


In [20]:
print("NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION-NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION")

NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION-NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION


In [21]:
#EVALUACIJA NOVELTY
def novelty(predicted: dict, pop: dict, u: int, n: int) -> (float, list):
    
    mean_self_information = []
    k = 0
    for pred100 in predicted: 
        self_information = 0
        k += 1
        for track_index in predicted[pred100].index:
            self_information += np.sum(-np.log2(pop[track_index]/u))
        mean_self_information.append(self_information/n)
    novelty = sum(mean_self_information)/k
    return novelty, mean_self_information
    
test_users = len(playlist_tracks_df) # Total unique users from notebook demo
test_recs_per_user = 100
# WHEN metrics.novelty is run
novelty_score, _ = novelty(
            predicted = preds_dict_rec_100,#predicted data
            pop = items_count_dict, #test data
            u = test_users, #broj korisnika
            n = test_recs_per_user #broj testnih recorda
)

print("NOVELTY SCORE:   ",novelty_score)

NOVELTY SCORE:    6.838314647706512


In [22]:
#EVALUACIJA CATALOG COVERAGE
def catalog_coverage(catalog: dict) -> float:
    sum_non_zero=0
    for elem in catalog:
        if(catalog[elem]!=0):
            sum_non_zero+=1
    catalog_coverage = round(sum_non_zero/(len(catalog)*1.0),2)
    return catalog_coverage
catalog_coverage = catalog_coverage(
    catalog = items_count_dict, #test data
)

print("CATALOG COVERAGE SCORE:   ",catalog_coverage)


CATALOG COVERAGE SCORE:    0.07


In [23]:
def personalization(predicted: List[list]) -> float:

    def make_rec_matrix(predicted: List[list]) -> sp.csr_matrix:
        df = pd.DataFrame(data=predicted).reset_index().melt(
            id_vars='index', value_name='item',
        )
        df = df[['index', 'item']].pivot(index='index', columns='item', values='item')
        df = pd.notna(df)*1
        rec_matrix = sp.csr_matrix(df.values)
        return rec_matrix

    #create matrix for recommendations
    predicted = np.array(predicted)
    rec_matrix_sparse = make_rec_matrix(predicted)

    #calculate similarity for every user's recommendation list
    similarity = cosine_similarity(X=rec_matrix_sparse, dense_output=False)

    #calculate average similarity
    dim = similarity.shape[0]
    personalization = (similarity.sum() - dim) / (dim * (dim - 1))
    return 1-personalization

pred100_list = [preds_dict_rec_100[playlist].index.tolist() for playlist in preds_dict_rec_100]

# WHEN metrics.personalization is run
personalization_score =personalization(
    predicted=pred100_list)

# THEN the personalization score should be within the expected value
print("PERSONALIZATION SCORE:    ",personalization_score)

PERSONALIZATION SCORE:     0.23293794099163367


In [24]:
def jaccard_similarity(dframe, average_100th):
    index_rating = dframe[dframe['rating']==0].index
    index_pred = dframe[dframe['pred']>average_100th].index
    intersection_size = len(index_rating[index_rating.isin(index_pred)])
    union_size = len(index_rating.union(index_pred))
    return intersection_size / union_size if union_size != 0 else 0.0

def inter_list_diversity(dframe, average_100th):
    length_frame = len(dframe)
    pairwise_similarities = []    
    similarity = jaccard_similarity(dframe, average_100th)    
    return similarity

diversity_score = inter_list_diversity(ui_rating_prediction_df,average_100th)
print("Inter-List Diversity Score:", diversity_score)

Inter-List Diversity Score: 0.013408274261426394
